In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap
from sklearn import neighbors, model_selection, metrics
from sklearn.inspection import DecisionBoundaryDisplay

In [83]:
data_folder = "data/traces/"

def read_date(date: str):
    """Aggregate data of a single from parts"""
    data = pd.DataFrame()

    for i in range(50):
        try:
            frame = pd.read_csv(data_folder + date + "-{}.csv".format(i))
            data = pd.concat([data, frame], axis=0)
        except OSError:
            pass

    return data


def read_date_part(date_part: str):
    """Read data of a part of a capture date"""
    data = pd.DataFrame()

    try:
        frame = pd.read_csv(data_folder + date_part + ".csv")
        data = pd.concat([data, frame], axis=0)
    except OSError:
        pass

    return data

data = read_date("16-09")
data.set_index("timestamp")

data

KeyboardInterrupt: 

In [75]:
def attach_window_id(timestamped_frame: pd.DataFrame) -> list[pd.DataFrame]:
    """Slice the dataframe into 1-second intervals"""
    first_packet_time = timestamped_frame.iloc[0]['timestamp']
    last_packet_time = timestamped_frame.iloc[-1]['timestamp']
    windowed_frame = timestamped_frame.set_index('timestamp', drop=False)

    ns_in_sec = int(1e9)
    number_of_windows = int((last_packet_time - first_packet_time) / ns_in_sec) + 1

    try:
        windowed_frame.insert(0, value=0, column="window_id")
    except Exception as e:
        print("It seems like the column 'window_id' already exists. Just updating values...")
        print(e)
    finally:
        windowed_frame.window_id.astype(np.int64)

    for window_id in range(0, number_of_windows):
        from_time = first_packet_time + window_id * ns_in_sec
        to_time = first_packet_time + (window_id  + 1) * ns_in_sec
        windowed_frame.loc[from_time:to_time, 'window_id'] = window_id

    windowed_frame.set_index('window_id', inplace=True)
    return windowed_frame


def attach_window_id_fast(timestamped_frame: pd.DataFrame) -> pd.DataFrame:
    first_packet_time = timestamped_frame.iloc[0]['timestamp']
    last_packet_time = timestamped_frame.iloc[-1]['timestamp']

    ns_in_sec = int(1e9)
    windows = range(first_packet_time - 1, last_packet_time + ns_in_sec, ns_in_sec)

    windowed_frame = pd.cut(timestamped_frame['timestamp'], windows, labels=range(len(windows) - 1)).to_frame()
    windowed_frame.columns = ["window_id"]
    return pd.concat([timestamped_frame, windowed_frame], axis=1).set_index('window_id')


devices = pd.read_csv("data/list_of_devices.csv")
windowed_frame = attach_window_id_fast(data)

windowed_frame

Slicing done!
CPU times: total: 3.34 s
Wall time: 3.63 s


In [ ]:
def generate_features_labeled(windowed_frame: pd.DataFrame, device_frame: pd.DataFrame):
    """Calculate the mode, mean, and median of a window"""
    features = pd.DataFrame()

    for device_index in device_frame.index:
        device = device_frame.iloc[device_index]
        device_packets = windowed_frame.loc[windowed_frame['eth_src'] == device['eth_src']]
        device_features = generate_features(device_packets)

        device_features.insert(len(device_features.columns), "device_name", device['device_name'])
        device_features.insert(len(device_features.columns), "iot", device['iot'])

        features = pd.concat([features, device_features])

    features.columns = ["mean", "std", "n_bytes", "device_name", "iot"]
    return features

def generate_features(windowed_frame: pd.DataFrame):
    window_groups = windowed_frame.groupby("window_id", observed=True)

    mean = window_groups['packet_size'].mean()
    std = window_groups['packet_size'].std()
    n_bytes = window_groups['packet_size'].sum()

    # TODO Not sure if fillna is a good idea
    return pd.concat([mean, std, n_bytes], axis=1).fillna(0)

def get_devices_in_window(windowed_frame: pd.DataFrame):
    devices_in_window = pd.DataFrame()

    for window_id in windowed_frame.index.unique():
        devices = windowed_frame.loc[window_id]['device_name']
        # iot = windowed_frame.loc[window_id]['iot']

        if not isinstance(devices, str):
            devices = devices.unique()

        devices_in_window = pd.concat([devices_in_window, pd.DataFrame({"window_id": window_id, "devices": devices})])

    return devices_in_window

labeled_features = generate_features_labeled(windowed_frame, devices)

labeled_features

In [ ]:
import sklearn.svm as svm
import sklearn.neighbors as neighbors
import sklearn.tree as tree
import sklearn.ensemble as ensemble

def train_and_test_classifiers(labeled_features, target_label, n_splits=10, shuffle=False, random_state=None):
    """
    Trains and tests a series of models on the given, labeled data
    (Knn, RF, DT, SVM, Majority Voting)
    """

    X = labeled_features.iloc[:, 1:3]
    y = labeled_features.iloc[:][target_label]
    kf = model_selection.StratifiedKFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    models = {
        "Knn" : neighbors.KNeighborsClassifier(n_neighbors=5, weights="distance"),
        "SVM" : svm.SVC(),
        "DT"  : tree.DecisionTreeClassifier(),
        "RF"  : ensemble.RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
    }
    models["Majority Voting"] = ensemble.VotingClassifier([ pair for pair in models.items() ], voting='hard')

    print("Training and testing models on {}".format(target_label))
    for (name, model) in models.items():
        accuracy = 0.0

        for (train_index, test_index) in kf.split(X, y):
            model.fit(X.iloc[train_index], y.iloc[train_index])

            accuracy += metrics.accuracy_score(y.iloc[test_index], model.predict(X.iloc[test_index]))

        print("{} mean accuracy over {} splits: {}".format(name, n_splits, accuracy/n_splits))

    return models

iot_classfiers = train_and_test_classifiers(labeled_features, 'iot')
device_classifiers = train_and_test_classifiers(labeled_features.loc[labeled_features['iot'] == True], 'device_name')

In [ ]:
def test_realistic(iot_classifier, device_classifier, unlabeled_features, devices_in_window):
    assert len(unlabeled_features) == len(devices_in_window)

    # accuracy = metrics.accuracy_score(devices_in_window, iot_classifier.predict(unlabeled_features))
    accuracy = metrics.accuracy_score(devices_in_window, device_classifier.predict(unlabeled_features))
    print("Accuracy:", accuracy)


unlabeled_features = generate_features(windowed_frame)
devices_in_window = get_devices_in_window(windowed_frame)

test_realistic(iot_classifier, device_classifier, unlabeled_features, devices_in_window)

In [82]:
def mode_mean_med(traffic_frame: pd.DataFrame):
    """Calculate the mode, mean, and median of a window"""

    # Keep only the mac address name and package size
    df = traffic_frame.loc[:, ['window_id' ,'eth_src', 'packet_size']]

    # Group the dataframe by window_id and mac adress & Calculate the statistical measures
    grouped = df.groupby(['window_id', 'eth_src']).agg({'packet_size': ['mean', 'median', lambda x: x.mode().iat[0]]})

    # Reset index to make window_id a column again
    grouped = grouped.reset_index()

    # Save the results
    grouped.to_csv('stats.cvs', index=False)

    return grouped

result = mode_mean_med(windowed_frame)


KeyboardInterrupt: 

In [81]:
%%time

def mode_mean_med_without_window(df: pd.DataFrame, devices: pd.DataFrame):
    """Calculate the mode, mean, and median of a window"""

    # Keep only the device name and mac address
    device_names = devices.loc[:, ['device_name', 'eth_src']]

    # Keep only the mac address name and package size
    df_size = df.loc[:, ['eth_src', 'packet_size']]

    # Merge the stats with the devices names
    merge_df = device_names.merge(df_size, on='eth_src')

    # Group the merged frame and calculate the stats
    stats = merge_df.groupby(['device_name']).agg({'packet_size': ['mean', 'median', lambda x: x.mode().iat[0]]})

    # Save the results
    stats.to_csv('stats_no_window.cvs', index=False)

    print(stats)

    return stats

devices = pd.read_csv("./data/list_of_devices.csv")
result_no_window = mode_mean_med_without_window(data, devices)


                                    packet_size                   
                                           mean  median <lambda_0>
device_name                                                       
Amazon Echo                          118.734701    75.0         66
Android Phone                        167.706915    66.0         66
Belkin Wemo switch                   415.264227   118.0         66
Belkin wemo motion sensor            113.394691    66.0         66
Blipcare Blood Pressure meter        108.393701    59.0         54
Dropcam                              206.533368   156.0        156
HP Printer                           168.309073    60.0        140
IPhone                               109.954947    66.0         54
Insteon Camera                       104.441343    90.0        102
Laptop                               111.325552    54.0         54
Light Bulbs LiFX Smart Bulb           96.446889    92.0        123
MacBook                              128.950355    66.0       

In [78]:
# n_neighbors = 5
#
# for metric in mmm_frame.columns[2:5]:
#     X = mmm_frame[metric].values
#     y = mmm_frame.iloc[:, -1].values
#     X = X.reshape(-1, 1)
#
#     X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)
#
#     for weights in ["uniform", "distance"]:
#         # we create an instance of Neighbours Classifier and fit the data.
#         clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
#         clf.fit(X_train, y_train)
#
#         accuracy = metrics.accuracy_score(y_test, clf.predict(X_test))
#         print("Accuracy of {} with {}: {}".format(metric, weights, accuracy))